### DATA 302 Midterm - Spring 2025
#### Due Friday, March 21 by 11:59pm

- These questions are based on the Billboard song database we've been working with in class.
- Submit your SQL code as a .data302 file. Make sure to review Checkpoint 2; files submitted in an incorrect format will receive a 0.
- Along those lines, make sure that your database table and column names are identical to what is defined in our ERD.

In [86]:
import billboard

import json

from collections import defaultdict as dd

from dbga import validate_submission_format

db = billboard.BillboardDB()

#where my answers go/formatted
my_sql = dd(dict)

setup = []

my_sql['setup'] = setup



---

1) `count(*)` vs `count(column_name)`

`count(*)` returns a count of the rows in your results, whereas `count(column_name)` returns the count of non-null entries in the column. Therefore, if a column contains null values, `count(*)` will return a greater number than `count(column_name)`.

Write a query that counts all records on tRating as well as the count of non-null entries in the rating column.

Return a single row containing:

- Number of records on tRating
- Number of non-null ratings

In [87]:
sql1 = """
SELECT 
    COUNT(*) AS total_records, 
    COUNT(rating) AS non_null_ratings
FROM 
    tRating;
"""
db.run_query(sql1)


,total_records,non_null_ratings
0,24092,5307


In [88]:
my_sql['q1']['sql'] = sql1

---

2) The date_entered field contains many entries for the same dates. How many unique dates are in this field?

Have your query return a single number; the number of unique dates in the date_entered field.

In [89]:
sql2 = """
SELECT 
    COUNT(DISTINCT date_entered) AS unique_dates
FROM 
    tRating;
"""
#have the row number disappear as well
db.run_query(sql2)


,unique_dates
0,68


In [90]:
my_sql['q2']['sql'] = sql2


---

3) Find the earliest date_entered, and return all songs that entered the charts on that date along with their rating for week 1.

Return:

- artist
- track
- date_entered
- rating (for week 1)

Order the results by song_id, ascending.

_Hint: No need to do anything fancy with the dates. They are formatted like YYYY-MM-DD which means that the minimum date is simply the first one when sorted lexicographically._

In [91]:
sql3 = """
SELECT 
    s.artist, 
    s.track, 
    r.date_entered, 
    r.rating AS rating
FROM 
    tRating r
JOIN 
    tSong s ON r.song_id = s.song_id
WHERE 
    r.date_entered = (SELECT MIN(date_entered) FROM tRating)
ORDER BY 
    r.song_id ASC;
"""
#i think the song charted for many times within the week which is why it repeats 
db.run_query(sql3)


,artist,track,date_entered,rating
0,Lonestar,Amazed,1999-06-05,81.0
1,Lonestar,Amazed,1999-06-05,54.0
2,Lonestar,Amazed,1999-06-05,44.0
3,Lonestar,Amazed,1999-06-05,39.0
4,Lonestar,Amazed,1999-06-05,38.0
...,...,...,...,...
71,Lonestar,Amazed,1999-06-05,NaN
72,Lonestar,Amazed,1999-06-05,NaN
73,Lonestar,Amazed,1999-06-05,NaN
74,Lonestar,Amazed,1999-06-05,NaN


In [92]:
my_sql['q3']['sql'] = sql3


---

4) Get a list of songs that were never in the top 50 (and remember that lower numbers are higher ratings!). Along with this, compute their average rating for the time they were on the charts.

Return:

- artist
- track
- average rating

Order your results by song_id, ascending (note that you can order by a column without returning the column in the results).

In [93]:
sql4 = """
SELECT 
    s.artist, 
    s.track, 
    AVG(r.rating) AS average_rating
FROM 
    tRating r
JOIN 
    tSong s ON r.song_id = s.song_id
WHERE 
    r.song_id NOT IN (
        SELECT song_id
        FROM tRating
        WHERE rating < 50
    )
GROUP BY 
    r.song_id
ORDER BY 
    r.song_id ASC;
"""
#so here it returns songs only higher than 50, not equal
db.run_query(sql4)


,artist,track,average_rating
0,2 Pac,Baby Don't Cry (Keep...,85.428571
1,2Ge+her,The Hardest Part Of ...,90.000000
2,3 Doors Down,Loser,67.100000
3,A*Teens,Dancing Queen,97.000000
4,"Adams, Yolanda",Open My Heart,67.750000
...,...,...,...
136,"Wright, Chely",It Was,77.300000
137,Yankee Grey,Another Nine Minutes,83.125000
138,"Yearwood, Trisha",Real Live Woman,84.166667
139,Ying Yang Twins,Whistle While You Tw...,88.857143


In [94]:
my_sql['q4']['sql'] = sql4


---

5) Write a query that returns all songs for artists who have more than 3 songs in the data.

Return:

- artist
- track

Order by artist, track, both ascending

In [97]:
sql5 = """
WITH TopN AS (
    SELECT artist
    FROM tSong
    GROUP BY artist
    HAVING COUNT(song_id) > 3
)
SELECT s.artist, s.track
FROM tSong s
JOIN TopN AS multiple_songs ON s.artist = multiple_songs.artist
ORDER BY s.artist ASC, s.track ASC;
"""

#now using TopN CTE for better clarity
db.run_query(sql5)


,artist,track
0,"Dixie Chicks, The",Cold Day In July
1,"Dixie Chicks, The",Cowboy Take Me Away
2,"Dixie Chicks, The",Goodbye Earl
3,"Dixie Chicks, The",Without You
4,"Houston, Whitney",Could I Have This Ki...
5,"Houston, Whitney",I Learned From The B...
6,"Houston, Whitney",My Love Is Your Love
7,"Houston, Whitney","Same Script, Differe..."
8,Jay-Z,Anything
9,Jay-Z,Big Pimpin'


In [98]:
my_sql['q5']['sql'] = sql5


---

6) Write a query that returns the number of weeks each song was on the chart (i.e., weeks with a non-null rating).

Return

- artist
- track
- number of weeks on the chart

Order by song_id ascending.

In [99]:
sql6 = """
SELECT s.artist, s.track, COUNT(r.week) AS number_of_weeks
FROM tSong s
JOIN tRating r ON s.song_id = r.song_id
WHERE r.rating IS NOT NULL
GROUP BY s.song_id
ORDER BY s.song_id ASC;
"""

# shows how many weeks each song was charted for
db.run_query(sql6)


,artist,track,number_of_weeks
0,2 Pac,Baby Don't Cry (Keep...,7
1,2Ge+her,The Hardest Part Of ...,3
2,3 Doors Down,Kryptonite,53
3,3 Doors Down,Loser,20
4,504 Boyz,Wobble Wobble,18
...,...,...,...
312,Yankee Grey,Another Nine Minutes,8
313,"Yearwood, Trisha",Real Live Woman,6
314,Ying Yang Twins,Whistle While You Tw...,14
315,Zombie Nation,Kernkraft 400,2


In [100]:
my_sql['q6']['sql'] = sql6


---

7) Write a query that returns all songs that were only on the chart for a single week.

Return

- artist
- track
- week
- rating

Order by song_id, ascending.

In [101]:
sql7 = """
SELECT s.artist, s.track, r.week, r.rating
FROM tSong s
JOIN tRating r ON s.song_id = r.song_id
WHERE s.song_id IN (
    SELECT song_id
    FROM tRating
    WHERE rating IS NOT NULL
    GROUP BY song_id
    HAVING COUNT(rating) = 1
) AND r.rating IS NOT NULL
ORDER BY s.song_id ASC;
"""
#songs that appear for only one week, ironcially also is week 1
#filter songs to see which had onyl one non-null-value
db.run_query(sql7)


,artist,track,week,rating
0,"Estefan, Gloria",No Me Dejes De Quere...,1,77
1,Fragma,Toca's Miracle,1,99
2,Ghostface Killah,Cherchez LaGhost,1,98
3,Master P,Souljas,1,98


In [102]:
my_sql['q7']['sql'] = sql7


---

8) Although the week numbers range up to 76, there are no songs that stayed on the charts the full 76 weeks. Determine the maximum number of weeks a song had non-null ratings, and return a list of all songs that have this many weeks of non-null ratings.

Return

- artist
- track
- number of weeks with ratings

Order by song_id, ascending.

In [103]:
sql8 = """
WITH SongWeeks AS (
    SELECT s.song_id, s.artist, s.track, COUNT(r.week) AS num_weeks
    FROM tSong s
    JOIN tRating r ON s.song_id = r.song_id
    WHERE r.rating IS NOT NULL
    GROUP BY s.song_id
)
SELECT artist, track, num_weeks
FROM SongWeeks
WHERE num_weeks = (SELECT MAX(num_weeks) FROM SongWeeks)
ORDER BY song_id ASC;
"""
#the only song that came up was Creed - Higher
db.run_query(sql8)

,artist,track,num_weeks
0,Creed,Higher,57


In [104]:
my_sql['q8']['sql'] = sql8


---

9) Some songs dip below the top 100 and then come back. When this happens there will be some weeks with null ratings, followed by weeks with non-null ratings. Find all songs where the total number of weeks with non-null ratings is not equal to their last week with a non-null rating.

Return:

- artist
- track
- total number of non-null ratings
- maximum week with a non-null rating

Order the results by song_id, ascending.

In [105]:
sql9 = """
SELECT s.artist, s.track, 
       COUNT(r.rating) AS non_null_ratings, 
       MAX(r.week) AS max_week_with_rating
FROM tSong s
JOIN tRating r ON s.song_id = r.song_id
WHERE r.rating IS NOT NULL
GROUP BY s.song_id
ORDER BY s.song_id ASC;
"""

#here, it shows the songs and how many weeks that they had ratings and the last week in which they were rated!
db.run_query(sql9)


,artist,track,non_null_ratings,max_week_with_rating
0,2 Pac,Baby Don't Cry (Keep...,7,7
1,2Ge+her,The Hardest Part Of ...,3,3
2,3 Doors Down,Kryptonite,53,53
3,3 Doors Down,Loser,20,20
4,504 Boyz,Wobble Wobble,18,18
...,...,...,...,...
312,Yankee Grey,Another Nine Minutes,8,8
313,"Yearwood, Trisha",Real Live Woman,6,6
314,Ying Yang Twins,Whistle While You Tw...,14,14
315,Zombie Nation,Kernkraft 400,2,2


In [109]:
my_sql['q9']['sql'] = sql9

In [110]:
submission_name = 'Midterm.data302'


with open(submission_name, 'w') as f:
    json.dump(my_sql, f)

In [111]:
path_db = 'databases/billboard.sqlite'

validate_submission_format(path_db,          # Path to the database
                           submission_name,  # Path to your .data302 file
                           num_questions=9   # Number of questions on this assignment
                          )

Report saved to:  VALIDATION_Midterm.txt
